In [1]:
import deepdoctection as dd
from pathlib import Path
import spacy
import re

config_overwrite = ["LANGUAGE='nld'",
                    "TEXT_ORDERING.INCLUDE_RESIDUAL_TEXT_CONTAINER=True",
                    "USE_PDF_MINER=True"]
analyzer = dd.get_dd_analyzer(config_overwrite=config_overwrite)
nlp = spacy.load("nl_core_news_sm")

def ocr_pdf(path):
    df = analyzer.analyze(path=Path.cwd() / path)

    doc = iter(df)

    return next(doc)


/home/marty/miniconda3/envs/PDFair/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[0403 09:28.44 @dd.py:422]  INF  Config: 
 {'DEVICE': 'cpu',
 'LANGUAGE': 'nld',
 'LAYOUT_NMS_PAIRS': {'COMBINATIONS': None, 'PRIORITY': None, 'THRESHOLDS': None},
 'LIB': 'PT',
 'OCR': {'CONFIG': {'TESSERACT': 'dd/conf_tesseract.yaml'},
         'USE_DOCTR': False,
         'USE_TESSERACT': True,
         'USE_TEXTRACT': False,
         'WEIGHTS': {'DOCTR_RECOGNITION': {'PT': 'doctr/crnn_vgg16_bn/pt/crnn_vgg16_bn-9762b0b0.pt',
                                           'TF': 'doctr/crnn_vgg16_bn/tf/crnn_vgg16_bn-76b7f2c6.zip'},
                     'DOCTR_WORD': {'PT': 'doctr/db_resnet50/pt/db_resnet50-ac60cadc.pt',
                                    'TF': 'doctr/db_resnet50/tf/db_resnet50-adcafc63.zip'}}},
 'PT': {'CELL

In [13]:
doc = ocr_pdf("BeslisnotaPDFs/nl.mnre1010.2e-b.2024.3.doc.1.pdf")


[0403 09:37.55 @doctectionpipe.py:84]  INF  Processing nl.mnre1010.2e-b.2024.3.doc.1_0.pdf


[0403 09:38.05 @context.py:126]  INF  ImageLayoutService total: 8.3715 sec.
[0403 09:38.05 @context.py:126]  INF  SubImageLayoutService total: 0.0001 sec.
[0403 09:38.05 @context.py:126]  INF  SubImageLayoutService total: 0.0002 sec.
[0403 09:38.05 @context.py:126]  INF  TableSegmentationService total: 0.0001 sec.
[0403 09:38.05 @context.py:126]  INF  TableSegmentationRefinementService total: 0.0 sec.
[0403 09:38.05 @context.py:126]  INF  TextExtractionService total: 0.0803 sec.
[0403 09:38.05 @context.py:126]  INF  TextExtractionService total: 0.0001 sec.
[0403 09:38.05 @context.py:126]  INF  MatchingService total: 0.0016 sec.
[0403 09:38.05 @context.py:126]  INF  TextOrderService total: 0.0102 sec.


In [14]:
def tokenize(text):
    BANNED_TOKENS = {"©", "\uf0b7"}

    tokens = [token.lower_ for token in nlp.make_doc(re.sub('\n+', ' ', text)) if not token.is_punct and token.text not in BANNED_TOKENS]

    return tokens


In [ ]:
for layout in sorted(doc.layouts, key=lambda x: x.reading_order):
    print(f"{layout.reading_order}: {tokenize(layout.text)}")


1: ['aan', 'de', 'minister-president']
2: ['kabinet', 'minister-president']
3: ['datum']
4: ['15', 'januari', '2024']
5: ['onze', 'referentie']
6: ['4381204']
7: ['beslisnota']
8: ['beantwoording', 'kamervragen', 'van', 'het', 'lid', 'palmen']
9: ['lijnparaaf', 'medeparaaf']
10: ['aanleiding', 'kamervragen', 'van', 'het', 'lid', 'palmen', 'nieuw', 'sociaal', 'contract', 'over', 'de', 'opvolging', 'van', 'de', 'aanbevelingen', 'bij', 'het', 'rapport', 'de', 'archivering', 'van', 'chatberichten', 'bij', 'het', 'ministerie', 'van', 'algemene', 'zaken', 'd.d.', '4', 'januari', '2024', 'kenmerk', '2024z00038']
11: ['geadviseerd', 'besluit']
12: ['ondertekenen', 'en', 'instemmen', 'met', 'verzending']
13: ['toelichting']
14: ['de', 'rvd', 'is', 'akkoord']
15: ['pagina', '1', 'van', '1']


In [ ]:
path = "BeslisnotaPDFs/nl.mnre1010.2e-b.2024.3.doc.1.pdf"
!pdftotext {path} -nopgbrk -l 1 -layout

with open("BeslisnotaPDFs/nl.mnre1010.2e-b.2024.3.doc.1.txt") as file:
    pdftotext = file.read()

pdftotext


'                                                                                Kabinet Minister-President\nAan de Minister-President\n                                                                                Datum\n                                                                                15 januari 2024\n\n                                                                                Onze referentie\n                                                                                4381204\n\n\n\n\nBeslisnota\n                        Beantwoording Kamervragen van het lid Palmen\n\n\n\nLijnparaaf              Medeparaaf\n\n\n\n\nAanleiding\nKamervragen van het lid Palmen (Nieuw Sociaal Contract) over de opvolging van\nde aanbevelingen bij het rapport “De archivering van chatberichten bij het\nMinisterie van Algemene Zaken” d.d. 4 januari 2024, kenmerk 2024Z00038.\n\nGeadviseerd besluit\nOndertekenen en instemmen met verzending.\n\nToelichting\n-  De RVD is akkoord.\n\n\n\n\n

In [ ]:
pdf = nlp(pdftotext)


In [ ]:
for i, s in enumerate(pdf.sents):
    print(f"{i}: {s}")


0: Kabinet Minister-President


1: Aan de Minister-President
Datum
15 januari 2024
Onze referentie
4381204

Beslisnota

2: Beantwoording
3: Kamervragen van het lid Palmen


4: Lijnparaaf


5: Medeparaaf

Aanleiding
Kamervragen van het lid
6: Palmen (Nieuw Sociaal Contract)
7: over de opvolging van
de aanbevelingen bij het rapport
8: “
9: De archivering van chatberichten bij het
Ministerie van Algemene Zaken
10: ” d.d. 4 januari 2024, kenmerk 2024Z00038.

11: Geadviseerd besluit
Ondertekenen en instemmen met verzending.
Toelichting

12: De RVD is akkoord.


13: Pagina 1 van 1


